## PART II: Hyper parameter tuning
So in this part we will use Sagemaker's Hyper parameter optimization framework to tune our XGBoost Model. But, we will try to keep the workflow the same as previous example. To achieve this, lets create a simple workflow that adapts the inputs to match the format of Sagemaker and adapts the output of sagemaker to match the format of the example workflow.

### Sagemaker integrations into Flyte - E.g. SagemakerXgBoostOptimizer
Working with AWS we have developed a new plugin (more coming soon) to integrate various functionalities of Sagemaker directly into Flyte. Flyte makes it easy to orchestrate across different compute fabrics. One of them is Sagemaker.

The example below shows a pre-created task that is available in flytekit, that allows using Sagemaker to tune XGBoost models.

In [1]:
from flytesagemakerplugin.sdk.tasks.plugin import SagemakerXgBoostOptimizer

xgtrainer_task = SagemakerXgBoostOptimizer(
    region="us-east-2",
    role_arn="arn:aws:iam::236416911133:role/SageMaker-Executor",
    resource_config={
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 25,
    },
    stopping_condition={"MaxRuntimeInSeconds": 43200, "MaxWaitTimeInSeconds": 43200},
    algorithm_specification={"TrainingInputMode": "File", "AlgorithmName": "xgboost"},
    retries=2,
    cacheable=True,
    cache_version="2.0",
)

{'Region': 'us-east-2', 'RoleArn': 'arn:aws:iam::236416911133:role/SageMaker-Executor', 'AlgorithmSpecification': {'TrainingImage': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:1', 'TrainingInputMode': 'File', 'AlgorithmName': 'xgboost'}, 'ResourceConfig': {'InstanceType': 'ml.m4.xlarge', 'InstanceCount': '1', 'VolumeSizeInGB': '25'}, 'StoppingCondition': {'MaxRuntimeInSeconds': '43200', 'MaxWaitTimeInSeconds': '43200'}}


### Sagemaker needs data in a specific format, so we can write a workflow to transform its inputs and outputs
So let us create a pipeline that
`Massages the input structured data frame into a format that Sagemaker likes`
 -> `Performs Model tuning with Sagemaker`
 -> `Untars model so that it matches our predict function`

In [3]:
from flytekit.sdk.workflow import workflow_class, Output, Input
from flytekit.sdk.types import Types
from workflows.sagemaker_xgboost_hpo import example_hyperparams, xgtrainer_task, convert_to_sagemaker_csv, untar_xgboost

@workflow_class
class StructuredSagemakerXGBoostHPO(object):
    # Input parameters
    static_hyperparameters = Input(
        Types.Generic,
        help="A list of the static hyperparameters to pass to the training jobs.",
        default=example_hyperparams,
    )
    train_data = Input(
        Types.Schema(), help="A Columnar schema that contains all the features used for training.",
    )
    train_target = Input(
        Types.Schema(), help="A Columnar schema that contains all the labeled results for train_data.",
    )

    validation_data = Input(
        Types.Schema(), help="A Columnar schema that contains all the features used for validation.",
    )
    validation_target = Input(
        Types.Schema(), help="A Columnar schema that contains all the labeled results for validation_data.",
    )

    sagemaker_transform = convert_to_sagemaker_csv(x_train=train_data, y_train=train_target,
                                                   x_test=validation_data, y_test=validation_target)

    # Node definitions
    train_node = xgtrainer_task(
        static_hyperparameters=static_hyperparameters,
        train=sagemaker_transform.outputs.train,
        validation=sagemaker_transform.outputs.validation,
    )

    untar = untar_xgboost(
        model_tar=train_node.outputs.model,
    )

    # Outputs
    model = Output(untar.outputs.model, sdk_type=Types.Blob)

# Create a launch plan that can be used in other workflows, with default inputs
fit_lp = StructuredSagemakerXGBoostHPO.create_launch_plan()

### We could run this Workflow on its own, but lets do something more interesting!